In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests
import warnings

warnings.filterwarnings("ignore")

# Setting up the plot style
plt.style.use("seaborn")
sns.set_context("paper", font_scale=1.2)

# Loading and cleaning data
data = pd.read_csv("export_ogama_csv_fitlered_3.csv")

# clean up column names (remove spaces head and tail)
data.columns = data.columns.str.strip()

print(data.columns)


# Filtering for Tasks 4, 5, 6
tasks = ["Task0", "Task1", "Task2", "Task3", "Task4", "Task5", "Task6"]
data = data[data["Trial: Name"].isin(tasks)]

# Mapping site codes to names
data["Site"] = data["Site"].map({"s": "smallpdf", "i": "ilovepdf"})

# Cleaning invalid values
data["Mouse: Left Clicks (count)"] = data["Mouse: Left Clicks (count)"].replace(
    -1, np.nan
)
data["Mouse: Pathlength (px)"] = data["Mouse: Pathlength (px)"].replace(0, np.nan)

# Grouping data by site and task
grouped = data.groupby(["Site", "Trial: Name"])

# Calculating summary statistics
summary_stats = grouped.agg(
    {
        "TrialDurSec": ["mean", "median", "std"],
        "Mouse: Left Clicks (count)": ["mean", "median", "std"],
        "Mouse: Pathlength (px)": ["mean", "median", "std"],
        "success": "mean",
    }
).round(2)

# Statistical tests
metrics = ["TrialDurSec", "Mouse: Left Clicks (count)", "Mouse: Pathlength (px)"]
p_values = []
comparisons = []

for task in tasks:
    for metric in metrics:
        smallpdf_data = data[
            (data["Site"] == "smallpdf") & (data["Trial: Name"] == task)
        ][metric].dropna()
        ilovepdf_data = data[
            (data["Site"] == "ilovepdf") & (data["Trial: Name"] == task)
        ][metric].dropna()

        # Checking normality
        if len(smallpdf_data) > 3 and len(ilovepdf_data) > 3:
            norm_smallpdf = stats.shapiro(smallpdf_data)[1]
            norm_ilovepdf = stats.shapiro(ilovepdf_data)[1]
            # Checking equal variances
            levene_stat, levene_p = stats.levene(smallpdf_data, ilovepdf_data)

            if norm_smallpdf > 0.05 and norm_ilovepdf > 0.05 and levene_p > 0.05:
                # T-test
                t_stat, p_val = stats.ttest_ind(smallpdf_data, ilovepdf_data)
            else:
                # Mann-Whitney U test
                t_stat, p_val = stats.mannwhitneyu(smallpdf_data, ilovepdf_data)
        else:
            t_stat, p_val = np.nan, np.nan

        p_values.append(p_val)
        comparisons.append(f"{task} - {metric}")

# Applying Bonferroni correction
reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method="bonferroni")
stat_results = pd.DataFrame(
    {
        "Comparison": comparisons,
        "P-Value": p_values,
        "Corrected P-Value": pvals_corrected,
        "Significant": reject,
    }
)

# Correlation analysis
corr_data = data[
    ["TrialDurSec", "Mouse: Left Clicks (count)", "Mouse: Pathlength (px)"]
].corr()

# Visualizations
fig = plt.figure(figsize=(15, 12))

# Bar charts for mean metrics
for i, metric in enumerate(metrics, 1):
    plt.subplot(3, 2, i)
    sns.barplot(x="Trial: Name", y=metric, hue="Site", data=data)
    plt.title(
        f"Mean {metric.replace('(count)', 'Count').replace('(px)', 'Path Length (px)')} by Task and Site"
    )
    plt.xlabel("Task")
    plt.ylabel(metric.replace("(count)", "Count").replace("(px)", "Path Length (px)"))
    plt.legend(title="Site")

# Box plots
plt.subplot(3, 2, 4)
sns.boxplot(x="Trial: Name", y="TrialDurSec", hue="Site", data=data)
plt.title("Distribution of Time-on-Task by Task and Site")
plt.xlabel("Task")
plt.ylabel("Time-on-Task (seconds)")
plt.legend(title="Site")

# Correlation heatmap
plt.subplot(3, 2, 5)
sns.heatmap(corr_data, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Between Metrics")

# Saving the plot
plt.tight_layout()
plt.savefig("ux_comparison.png")
plt.close()

# Saving summary statistics and test results
summary_stats.to_csv("summary_statistics.csv")
stat_results.to_csv("statistical_tests.csv")

# Printing interesting finding
significant = stat_results[stat_results["Significant"]]
if not significant.empty:
    print(
        "Interesting Finding: Significant differences found in the following comparisons:"
    )
    for _, row in significant.iterrows():
        print(
            f"- {row['Comparison']}: Corrected P-Value = {row['Corrected P-Value']:.4f}"
        )
else:
    print("No significant differences found after Bonferroni correction.")


Index(['Subject Id', 'Subject name', 'Subject Sex', 'Site', 'Trial: ID',
       'Trial: Sequence', 'Trial: Name', 'Trial: Starting Time (ms)',
       'Trial: Duration (ms)', 'Mouse: Left Clicks (count)',
       'Mouse: Left Clicks (count/s)', 'Mouse: Right Clicks (count)',
       'Mouse: Right Clicks (count/s)',
       'Mouse: Fixations Until First Click (count)',
       'Mouse: Time Until First Click (ms)', 'Mouse: Pathlength (px)',
       'Mouse: Path velocity (px/s)', 'Mouse: Average Saccade Velocity (px/s)',
       'TrialDurSec', 'success'],
      dtype='object')
No significant differences found after Bonferroni correction.
